In [1]:
from matplotlib import pyplot as plt
from mxnet import autograd, nd, init, gluon
from mxnet.gluon import data as gdata
from mxnet.gluon import loss as gloss
from mxnet.gluon import nn
from time import time
import d2lzh as d2l
import random
import sys 

# 05. 深度学习计算
卷积神经⽹络是近年来深度学习能在计算机视觉领域取得突破性成果的基⽯。它也逐渐在被其他诸如⾃然语⾔处理、推荐系统和语⾳识别等领域⼴泛使⽤。

我们将先描述卷积神经⽹络中卷积层和池化层的⼯作原理，并解释填充、步幅、输⼊通道和输出通道的含义。在掌握了这些基础知识以后，我们将探究数个具有代表性的深度卷积神经⽹络的设计思路。这些模型包括最早提出的AlexNet，以及后来的使⽤重复元素的⽹络(VGG)、⽹络中的⽹络(NiN)、含并⾏连结的⽹络(GoogLeNet)、残差⽹络(ResNet)和稠密连接⽹络(DenseNet)。它们中有不少在过去⼏年的ImageNet⽐赛中⼤放异彩。虽然深度模型看上去只是具有很多层的神经⽹络，然而获得有效的深度模型并不容易。有幸的是，本章阐述的批量归⼀化和残差⽹络为训练和设计深度模型提供了两类重要思路。

## 5.1 ⼆维卷积层
卷积神经⽹络是含有卷积层的神经⽹络。本章中介绍的卷积神经⽹络均使⽤最常⻅的⼆维卷积层。它有⾼和宽两个空间维度，常⽤来处理图像数据。

### 5.1.1 ⼆维互相关运算
虽然卷积层得名于卷积(convolution)运算，但我们通常在卷积层中使⽤更加直观的互相关(cross-correlation)运算。在⼆维卷积层中，⼀个⼆维输⼊数组和⼀个⼆维核(kernel)数组通过互相关运算输出⼀个⼆维数组。

我们⽤⼀个具体例⼦来解释⼆维互相关运算的含义。如`图5.1`所⽰，输⼊是⼀个⾼和宽均为3的⼆维数组。我们将该数组的形状记为`3 × 3`。核数组的⾼和宽分 别为2。该数组在卷积计算中⼜称卷积核或过滤器(filter)。卷积核窗口(⼜称卷积窗口)的形状取决于卷积核的⾼和宽，即`2 × 2`。`图5.1`中的阴影部分为第⼀个输出元素及其计算所使⽤的输⼊和核数组元素：`0 × 0 + 1 × 1 + 3 × 2 + 4 × 3 = 19`。

<img src="images/05_01.png" style="width:350px;"/>

在⼆维互相关运算中，卷积窗口从输⼊数组的最左上⽅开始，按从左往右、从上往下的顺序，依次在输⼊数组上滑动。当卷积窗口滑动到某⼀位置时，窗口中的输⼊⼦数组与核数组按元素相乘并求和，得到输出数组中相应位置的元素。

下⾯我们将上述过程实现在`corr2d`函数⾥：

In [2]:
# 本函数已保存在d2lzh包中⽅便以后使⽤
def corr2d(X, K):
    h, w = K.shape
    Y = nd.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum() 
    return Y

我们可以构造`图5.1`中的输⼊数组$X$、核数组$K$来验证⼆维互相关运算的输出。

In [3]:
X = nd.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = nd.array([[0, 1], [2, 3]])
corr2d(X, K)


[[19. 25.]
 [37. 43.]]
<NDArray 2x2 @cpu(0)>

### 5.1.2 ⼆维卷积层
⼆维卷积层将输⼊和卷积核做互相关运算，并加上⼀个标量偏差来得到输出。卷积层的模型参数包括了卷积核和标量偏差。在训练模型的时候，通常我们先对卷积核随机初始化，然后不断迭代卷积核和偏差。

下⾯基于`corr2d`函数来实现⼀个⾃定义的⼆维卷积层。在构造函数`__init__`⾥我们声明`weight`和`bias`这两个模型参数。前向计算函数`forward`则是直接调⽤`corr2d`函数再加上偏差。

In [4]:
class Conv2D(nn.Block):
    def __init__(self, kernel_size, **kwargs):
        super(Conv2D, self).__init__(**kwargs) 
        self.weight = self.params.get('weight', shape=kernel_size) 
        self.bias = self.params.get('bias', shape=(1,))

    def forward(self, x):
        return corr2d(x, self.weight.data()) + self.bias.data()

卷积窗口形状为$p×q$的卷积层称为$p×q$卷积层。同样，$p×q$卷积或$p×q$卷积核说明卷积核的⾼和宽分别为$p$和$q$。

### 5.1.3 图像中物体边缘检测
下⾯我们来看⼀个卷积层的简单应⽤：检测图像中物体的边缘，即找到像素变化的位置。⾸先我们构造⼀张$6 × 8$的图像。它中间4列为⿊，其余为⽩：

In [5]:
X = nd.ones((6, 8))
X[:, 2:6] = 0
X


[[1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]]
<NDArray 6x8 @cpu(0)>

然后我们构造⼀个⾼和宽分别为1和2的卷积核$K$。当它与输⼊做互相关运算时，如果横向相邻元素相同，输出为0；否则输出为⾮0。

In [6]:
K = nd.array([[1, -1]])
Y = corr2d(X, K) 
Y


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

由此，我们可以看出，卷积层可通过重复使⽤卷积核有效地表征局部空间。

### 5.1.4 通过数据学习核数组
最后我们来看⼀个例⼦，它使⽤物体边缘检测中的输⼊数据$X$和输出数据$Y$来学习我们构造的核数组$K$。

我们⾸先构造⼀个卷积层，将其卷积核初始化成随机数组。接下来在每⼀次迭代中，我们使⽤平⽅误差来⽐较$Y$和卷积层的输出，然后计算梯度来更新权重。简单起⻅，这⾥的卷积层忽略了偏差。

虽然我们之前构造了`Conv2D`类，但由于`corr2d`使⽤了对单个元素赋值(`[i, j]=`)的操作因而⽆法⾃动求梯度。下⾯我们使⽤`Gluon`提供的`Conv2D`类来实现这个例⼦。

In [7]:
# 构造⼀个输出通道数为1，核数组形状是(1, 2)的⼆维卷积层 
conv2d = nn.Conv2D(1, kernel_size=(1, 2)) 
conv2d.initialize()

# ⼆维卷积层使⽤4维输⼊输出，格式为(样本, 通道, ⾼, 宽)，这⾥批量⼤⼩和通道数均为1
X = X.reshape((1, 1, 6, 8)) 
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    with autograd.record():
        Y_hat = conv2d(X) 
        l = (Y_hat - Y) ** 2 
    l.backward() 
    # 简单起⻅，这⾥忽略了偏差 
    conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad() 
    if (i + 1) % 2 == 0:
        print('batch %d, loss %.3f' % (i + 1, l.sum().asscalar()))

batch 2, loss 4.949
batch 4, loss 0.831
batch 6, loss 0.140
batch 8, loss 0.024
batch 10, loss 0.004


可以看到，10次迭代后误差已经降到了⼀个⽐较小的值。现在来看⼀下学习到的核数组。

In [8]:
conv2d.weight.data().reshape((1, 2))


[[ 0.9895    -0.9873705]]
<NDArray 1x2 @cpu(0)>

### 5.1.5 互相关运算和卷积运算
实际上，卷积运算与互相关运算类似。

为了得到卷积运算的输出，我们只需将核数组左右翻转并上下翻转，再与输⼊数组做互相关运算。可⻅，卷积运算和互相关运算虽然类似，但如果它们使⽤相同的核数组，对于同⼀个输⼊，输出往往并不相同。

你也许会好奇卷积层为何能使⽤互相关运算替代卷积运算。其实，在深度学习中核数组都是学出来的：卷积层⽆论使⽤互相关运算或卷积运算都不影响模型预测时的输出。为了解释这⼀点，假设卷积层使⽤互相关运算学出`图5.1`中的核数组。设其他条件不变，使⽤卷积运算学出的核数组即`图5.1`中的核数组按上下、左右翻转。也就是说，`图5.1`中的输⼊与学出的已翻转的核数组再做卷积运算时，依然得到`图5.1`中的输出。为了与⼤多数深度学习⽂献⼀致，如⽆特别说明，本书中提到的卷积运算均指互相关运算。

### 5.1.6 特征图和感受野
⼆维卷积层输出的⼆维数组可以看作是输⼊在空间维度(宽和⾼)上某⼀级的表征，也叫`特征图` (feature map)。影响元素$x$的前向计算的所有可能输⼊区域(可能⼤于输⼊的实际尺⼨)叫做$x$的`感受野`(receptive field)。以`图5.1`为例，输⼊中阴影部分的四个元素是输出中阴影部分元素的感受野。

我们将`图5.1`中形状为$2 × 2$的输出记为$Y$，并考虑⼀个更深的卷积神经⽹络：将$Y$与另⼀个形状为$2 × 2$的核数组做互相关运算，输出单个元素$z$。那么，$z$在$Y$上的感受野包括$Y$的全部四个元素，在输⼊上的感受野包括其中全部9个元素。可⻅，我们可以通过更深的卷积神经⽹络使特征图中单个元素的感受野变得更加⼴阔，从而捕捉输⼊上更⼤尺⼨的特征。

我们常使⽤`元素`⼀词来描述数组或矩阵中的成员。在神经⽹络的术语中，这些元素也可称为`单元`。当含义明确时，本书不对这两个术语做严格区分。


## 5.2 填充和步幅
上⼀节我们使⽤⾼和宽为3的输⼊与⾼和宽为2的卷积核得到⾼和宽为2的输出。⼀般来说，假设输⼊形状是$n_h \times n_w$，卷积核窗口形状是$k_h \times k_w$，那么输出形状将会是
+ $(n_h − k_h + 1) \times (n_w − k_w + 1)$

所以卷积层的输出形状由输⼊形状和卷积核窗口形状决定。本节我们将介绍卷积层的两个超参数，即`填充`和`步幅`。它们可以对给定形状的输⼊和卷积核改变输出形状。

### 5.2.1 填充
`填充`(padding)是指在输⼊⾼和宽的两侧填充元素(通常是0元素)。`图5.2`⾥我们在原输⼊⾼和宽的两侧分别添加了值为0的元素，使得输⼊⾼和宽从3变成了5，并导致输出⾼和宽由2增加到4。

<img src="images/05_02.png" style="width:450px;"/>

一般来说，如果在高的两侧一共填充$p_h$行，在宽的两侧一共填充$p_w$列，那么输出形状将会是
+ $(n_h-k_h+p_h+1) \times (n_w-k_w+p_w+1)$

也就是说，输出的高和宽会分别增加$p_h$和$p_w$。

在很多情况下，我们会设置$p_h=k_h-1$和$p_w=k_w-1$来使输入和输出具有相同的高和宽。这样会方便在构造网络时推测每个层的输出形状。假设这里$k_h$是奇数，我们会在高的两侧分别填充$p_h/2$行。如果$k_h$是偶数，一种可能是在输入的顶端一侧填充$\lceil p_h/2\rceil$行，而在底端一侧填充$\lfloor p_h/2\rfloor$行。在宽的两侧填充同理。

卷积神经网络经常使用奇数高宽的卷积核，如1、3、5和7，所以两端上的填充个数相等。对任意的二维数组$X$，设它的第$i$行第$j$列的元素为$X[i,j]$。当两端上的填充个数相等，并使输入和输出具有相同的高和宽时，我们就知道输出$Y[i,j]$是由输入以$X[i,j]$为中心的窗口同卷积核进行互相关计算得到的。

In [9]:
# 定义⼀个函数来计算卷积层。它初始化卷积层权重，并对输⼊和输出做相应的升维和降维
def comp_conv2d(conv2d, X):
    conv2d.initialize() 
    # (1, 1)代表批量⼤⼩和通道数(`多输⼊通道和多输出通道`⼀节将介绍)均为1 
    X = X.reshape((1, 1) + X.shape) 
    Y = conv2d(X) 
    return Y.reshape(Y.shape[2:]) # 排除不关⼼的前两维：批量和通道

# 注意这⾥是两侧分别填充1⾏或列，所以在两侧⼀共填充2⾏或列
conv2d = nn.Conv2D(1, kernel_size=3, padding=1)
X = nd.random.uniform(shape=(8, 8))
comp_conv2d(conv2d, X).shape

(8, 8)

当卷积核的⾼和宽不同时，我们也可以通过设置⾼和宽上不同的填充数使输出和输⼊具有相同的⾼和宽。

In [10]:
# 使⽤⾼为5、宽为3的卷积核。在⾼和宽两侧的填充数分别为2和1 
conv2d = nn.Conv2D(1, kernel_size=(5, 3), padding=(2, 1)) 
comp_conv2d(conv2d, X).shape

(8, 8)

### 5.2.2 步幅
在二维互相关运算时，卷积窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。我们将每次滑动的行数和列数称为`步幅`(stride)。

`图5.3`展示了在高上步幅为3、在宽上步幅为2的二维互相关运算。可以看到，输出第一列第二个元素时，卷积窗口向下滑动了3行，而在输出第一行第二个元素时卷积窗口向右滑动了2列。当卷积窗口在输入上再向右滑动2列时，由于输入元素无法填满窗口，无结果输出。`图5.3`中的阴影部分为输出元素及其计算所使用的输入和核数组元素：
+ $0\times0+0\times1+1\times2+2\times3=8$
+ $0\times0+6\times1+0\times2+0\times3=6$

<img src="images/05_03.png" style="width:400px;"/>

一般来说，当高上步幅为$s_h$，宽上步幅为$s_w$时，输出形状为
+ $\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor$

如果设置$p_h=k_h-1$和$p_w=k_w-1$，那么输出形状将简化为$\lfloor(n_h+s_h-1)/s_h\rfloor \times \lfloor(n_w+s_w-1)/s_w\rfloor$。更进一步，如果输入的高和宽能分别被高和宽上的步幅整除，那么输出形状将是$(n_h/s_h) \times (n_w/s_w)$。

下面我们令高和宽上的步幅均为2，从而使输入的高和宽减半。

In [11]:
conv2d = nn.Conv2D(1, kernel_size=3, padding=1, strides=2) 
comp_conv2d(conv2d, X).shape

(4, 4)

接下来是⼀个稍微复杂点⼉的例⼦。

In [12]:
conv2d = nn.Conv2D(1, kernel_size=(3, 5), padding=(0, 1), strides=(3, 4)) 
comp_conv2d(conv2d, X).shape

(2, 2)

为了表述简洁，当输入的高和宽两侧的填充数分别为$p_h$和$p_w$时，我们称填充为$(p_h, p_w)$。特别地，当$p_h = p_w = p$时，填充为$p$。当在高和宽上的步幅分别为$s_h$和$s_w$时，我们称步幅为$(s_h, s_w)$。特别地，当$s_h = s_w = s$时，步幅为$s$。在默认情况下，填充为0，步幅为1。

## 5.3 多输入通道和多输出通道
前面两节里我们用到的输入和输出都是二维数组，但真实数据的维度经常更高。例如，彩色图像在高和宽2个维度外还有RGB 3个颜色通道。假设彩色图像的高和宽分别是$h$和$w$，那么它可以表示为一个$3\times h\times w$的多维数组。我们将大小为3的这一维称为`通道`(channel)维。

### 5.3.1 多输入通道
当输入数据含多个通道时，我们需要构造一个输入通道数与输入数据的通道数相同的卷积核，从而能够与含多通道的输入数据做互相关运算。

假设输入数据的通道数为$c_i$，那么卷积核的输入通道数同样为$c_i$。设卷积核窗口形状为$k_h\times k_w$：
+ 当$c_i=1$时，我们知道卷积核只包含一个形状为$k_h\times k_w$的二维数组
+ 当$c_i > 1$时，我们将会为每个输入通道各分配一个形状为$k_h\times k_w$的核数组

把这$c_i$个数组在输入通道维上连结，即得到一个形状为$c_i\times k_h\times k_w$的卷积核。由于输入和卷积核各有$c_i$个通道，我们可以在各个通道上对输入的二维数组和卷积核的二维核数组做互相关运算，再将这$c_i$个互相关运算的二维输出按通道相加，得到一个二维数组。这就是含多个通道的输入数据与多输入通道的卷积核做二维互相关运算的输出。

`图5.4`展示了含2个输入通道的二维互相关计算的例子。`图5.4`中阴影部分为第一个输出元素及其计算所使用的输入和核数组元素：
+ $(1\times1+2\times2+4\times3+5\times4)+(0\times0+1\times1+3\times2+4\times3)=56$

<img src="images/05_04.png" style="width:450px;"/>

实现含多个输⼊通道的互相关运算时，只需要对每个通道做互相关运算，然后通过`add_n`函数来进⾏累加。

In [13]:
def corr2d_multi_in(X, K):
    # ⾸先沿着X和K的第0维(通道维)遍历。然后使⽤*将结果列表变成`add_n`函数的位置参数 
    # 来进⾏相加 
    return nd.add_n(*[d2l.corr2d(x, k) for x, k in zip(X, K)])

我们可以构造`图5.4`中的输⼊数组$X4、核数组$K$来验证互相关运算的输出。

In [14]:
X = nd.array([
    [[0, 1, 2], [3, 4, 5], [6, 7, 8]], 
    [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
])

K = nd.array([
    [[0, 1], [2, 3]], 
    [[1, 2], [3, 4]]
])

corr2d_multi_in(X, K)


[[ 56.  72.]
 [104. 120.]]
<NDArray 2x2 @cpu(0)>

### 5.3.2 多输出通道
当输入通道有多个时，因为我们对各个通道的结果做了累加，所以不论输入通道数是多少，输出通道数总是为1。

设卷积核输入通道数和输出通道数分别为$c_i$和$c_o$，高和宽分别为$k_h$和$k_w$。如果希望得到含多个通道的输出，我们可以为每个输出通道分别创建形状为$c_i\times k_h\times k_w$的核数组。将它们在输出通道维上连结，卷积核的形状即$c_o\times c_i\times k_h\times k_w$。在做互相关运算时，每个输出通道上的结果由卷积核在该输出通道上的核数组与整个输入数组计算而来。

下面我们实现一个互相关运算函数来计算多个通道的输出。

In [15]:
def corr2d_multi_in_out(X, K):
    # 对K的第0维遍历，每次同输⼊X做互相关计算。所有结果使⽤stack函数合并在⼀起 
    return nd.stack(*[corr2d_multi_in(X, k) for k in K])

我们将核数组$K$同$K+1$($K$中每个元素加一)和$K+2$连结在一起来构造一个输出通道数为3的卷积核。

In [16]:
K = nd.stack(K, K + 1, K + 2) 
K.shape

(3, 2, 2, 2)

下⾯我们对输⼊数组$X$与核数组$K$做互相关运算。此时的输出含有3个通道。其中第⼀个通道的结果与之前输⼊数组X与多输⼊通道、单输出通道核的计算结果⼀致。

In [17]:
corr2d_multi_in_out(X, K)


[[[ 56.  72.]
  [104. 120.]]

 [[ 76. 100.]
  [148. 172.]]

 [[ 96. 128.]
  [192. 224.]]]
<NDArray 3x2x2 @cpu(0)>

### 5.3.3 1X1卷积层
最后我们讨论卷积窗口形状为$1\times 1$的多通道卷积层。我们通常称之为$1\times 1$卷积层，并将其中的卷积运算称为$1\times 1$卷积。

因为使用了最小窗口，$1\times 1$卷积失去了卷积层可以识别高和宽维度上相邻元素构成的模式的功能。实际上，$1\times 1$卷积的主要计算发生在通道维上。

`图5.5`展示了使用输入通道数为3、输出通道数为2的$1\times 1$卷积核的互相关计算。值得注意的是，输入和输出具有相同的高和宽。输出中的每个元素来自输入中在高和宽上相同位置的元素在不同通道之间的按权重累加。假设我们将通道维当作特征维，将高和宽维度上的元素当成数据样本，那么$1\times 1$卷积层的作用与全连接层等价。

<img src="images/05_05.png" style="width:600px;"/>

下⾯我们使⽤全连接层中的矩阵乘法来实现$1\times 1$卷积。这⾥需要在矩阵乘法运算前后对数据形状做⼀些调整。

In [18]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape 
    c_o = K.shape[0] 
    X = X.reshape((c_i, h * w)) 
    K = K.reshape((c_o, c_i)) 
    Y = nd.dot(K, X) # 全连接层的矩阵乘法 
    return Y.reshape((c_o, h, w))

经验证，做$1\times 1$卷积时，以上函数与之前实现的互相关运算函数corr2d_multi_in_out等价。

In [19]:
X = nd.random.uniform(shape=(3, 3, 3))
K = nd.random.uniform(shape=(2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K) 
Y2 = corr2d_multi_in_out(X, K)

(Y1 - Y2).norm().asscalar() < 1e-6

True

在之后的模型里我们将会看到$1\times 1$卷积层被当作保持高和宽维度形状不变的全连接层使用。于是，我们可以通过调整网络层之间的通道数来控制模型复杂度。

## 5.4 池化层
在`二维卷积层`一节里介绍的图像物体边缘检测应用中，我们构造卷积核从而精确地找到了像素变化的位置。设任意二维数组$X$的$i$行$j$列的元素为$X[i, j]$。如果我们构造的卷积核输出$Y[i, j]=1$，那么说明输入中$X[i, j]$和$X[i, j+1]$数值不一样。这可能意味着物体边缘通过这两个元素之间。但实际图像里，我们感兴趣的物体不会总出现在固定位置：即使我们连续拍摄同一个物体也极有可能出现像素位置上的偏移。这会导致同一个边缘对应的输出可能出现在卷积输出$Y$中的不同位置，进而对后面的模式识别造成不便。

池化层就是为了缓解卷积层对位置的过度敏感性。

### 5.4.1 二维最大池化层和平均池化层
同卷积层一样，池化层每次对输入数据的一个固定形状窗口(又称`池化窗口`)中的元素计算输出。不同于卷积层里计算输入和核的互相关性，池化层直接计算池化窗口内元素的最大值或者平均值。该运算也分别叫做`最大池化`或`平均池化`。在二维最大池化中，池化窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。当池化窗口滑动到某一位置时，窗口中的输入子数组的最大值即输出数组中相应位置的元素。

<img src="images/05_06.png" style="width:300px;"/>

`图5.6`展示了池化窗口形状为$2\times 2$的最大池化，阴影部分为第一个输出元素及其计算所使用的输入元素。

二维平均池化的工作原理是将最大运算符替换成平均运算符。

池化窗口形状为$p \times q$的池化层称为$p \times q$池化层，其中的池化运算叫作$p \times q$池化。

我们再次回到物体边缘检测的例子。现在我们将卷积层的输出作为$2\times 2$最大池化的输入。设该卷积层输入是$X$、池化层输出为$Y$。无论是$X[i, j]$和$X[i, j+1]$值不同，还是$X[i, j+1]$和$X[i, j+2]$不同，池化层输出均有$Y[i, j]=1$。也就是说，使用$2\times 2$最大池化层时，只要卷积层识别的模式在高和宽上移动不超过一个元素，我们依然可以将它检测出来。

In [20]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = nd.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean() 
    return Y

我们可以构造`图5.6`中的输⼊数组$X$来验证⼆维最⼤池化层的输出。

In [21]:
X = nd.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]]) 
pool2d(X, (2, 2))


[[4. 5.]
 [7. 8.]]
<NDArray 2x2 @cpu(0)>

同时我们实验⼀下平均池化层。

### 5.4.2 填充和步幅
池化层也可以在输入的高和宽两侧的填充并调整窗口的移动步幅来改变输出形状。池化层填充和步幅与卷积层填充和步幅的工作机制一样。我们将通过`nn`模块里的二维最大池化层`MaxPool2D`来演示池化层填充和步幅的工作机制。我们先构造一个形状为(1, 1, 4, 4)的输入数据，前两个维度分别是批量和通道。

In [22]:
X = nd.arange(16).reshape((1, 1, 4, 4))
X


[[[[ 0.  1.  2.  3.]
   [ 4.  5.  6.  7.]
   [ 8.  9. 10. 11.]
   [12. 13. 14. 15.]]]]
<NDArray 1x1x4x4 @cpu(0)>

默认情况下，`MaxPool2D`实例⾥步幅和池化窗口形状相同。形状为(3, 3)的池化窗口默认获得形状为(3, 3)的步幅。

In [23]:
pool2d = nn.MaxPool2D(3) 
pool2d(X)


[[[[10.]]]]
<NDArray 1x1x1x1 @cpu(0)>

我们可以⼿动指定步幅和填充。

In [24]:
pool2d = nn.MaxPool2D(3, padding=1, strides=2) 
pool2d(X)


[[[[ 5.  7.]
   [13. 15.]]]]
<NDArray 1x1x2x2 @cpu(0)>

我们也可以指定⾮正⽅形的池化窗口，并分别指定⾼和宽上的填充和步幅。

In [25]:
pool2d = nn.MaxPool2D((2, 3), padding=(1, 2), strides=(2, 3)) 
pool2d(X)


[[[[ 0.  3.]
   [ 8. 11.]
   [12. 15.]]]]
<NDArray 1x1x3x2 @cpu(0)>

### 5.4.3 多通道
在处理多通道输⼊数据时，池化层对每个输⼊通道分别池化，而不是像卷积层那样将各通道的输⼊按通道相加。这意味着池化层的输出通道数与输⼊通道数相等。

In [26]:
X = nd.concat(X, X + 1, dim=1) 
X


[[[[ 0.  1.  2.  3.]
   [ 4.  5.  6.  7.]
   [ 8.  9. 10. 11.]
   [12. 13. 14. 15.]]

  [[ 1.  2.  3.  4.]
   [ 5.  6.  7.  8.]
   [ 9. 10. 11. 12.]
   [13. 14. 15. 16.]]]]
<NDArray 1x2x4x4 @cpu(0)>

池化后，我们发现输出通道数仍然是2。

In [27]:
pool2d = nn.MaxPool2D(3, padding=1, strides=2) 
pool2d(X)


[[[[ 5.  7.]
   [13. 15.]]

  [[ 6.  8.]
   [14. 16.]]]]
<NDArray 1x2x2x2 @cpu(0)>